# Swap
Agent $\alpha$ swaps asset $i$ with the protocol for asset $j$.

## Sell
Agent $\alpha$ sells asset $i$ in quantity $\Delta R_i$ for asset $j$.

### Constraints

The swap will keep the swap invariant for assets $i$ and $j$, $R_iQ_i$ and $R_jQ_j$, constant, in the case that fees are set to $0$.

## Updating AMM state

### Calculating swaps

#### Case 1: Asset $j$ bought in quantity $-\Delta R_j$ with asset $i$

$$
\begin{align}
\tilde{\Delta Q_j} &= Q_j\frac{- \Delta R_j}{R_j(1 - f_A) + \Delta R_j}\\
\tilde{\Delta Q_i} &= -\frac{\tilde{\Delta Q_j}}{1 - f_P}\\
\Delta R_i &= R_i\frac{- \tilde{\Delta Q_i}}{Q_i + \tilde{\Delta Q_i}}
\end{align}
$$

Now that the correct $\Delta R_i$ is computed, we can treat this trade as a "sell" trade of asset $i$ in quantity $\Delta R_i$.

#### Case 2: Asset $i$ sold in quantity $\Delta R_i$ for asset $j$

Asset $i$ is sold for asset $j$ in a series of trades, with 
$$
\begin{align}
\Delta Q_i &= Q_i\frac{- \Delta R_i}{R_i + \Delta R_i}\\
\Delta Q_j' &= -\Delta Q_i (1 - f_P)\\
\Delta R_j' &= R_j\frac{- \Delta Q_j'}{Q_j + \Delta Q_j'}\\
\Delta R_j &= \Delta R_j'(1 - f_A)
\end{align}
$$

### Fee accounting

The LRNA fee amount is $-\Delta Q_i f_P$, and it is burned until $L$ is brought up to $0$, at which point fees are moved to the LRNA side of the HDX sub-pool in Omnipool.
$$
\begin{align}
\Delta L &= min(-\Delta Q_i f_P, -L)\\
\Delta Q_H &= -\Delta Q_i f_P - \Delta L
\end{align}
$$
where $Q_H$ is the LRNA side of the HDX sub-pool.



## Direct calculation (updating pool state)
### $\Delta R_i > 0$ provided
$$
\begin{align}
\Delta Q_i &= Q_i \frac{-\Delta R_i}{R_i + \Delta R_i}\\
\Delta Q_j &= -\Delta Q_i(1 - f_P)\\
\Delta R_j &= R_j \frac{-\Delta Q_j}{Q_j + \Delta Q_j}(1 - f_A)\\
\Delta L &= min(-\Delta Q_i f_P, -L)\\
\Delta Q_H &= -\Delta Q_i f_P - \Delta L
\end{align}
$$

### $\Delta R_j < 0$ provided
$$
\begin{align}
\Delta Q_j &= Q_j\frac{-\Delta R_j}{R_j(1 - f_A) + \Delta R_j}\\
\Delta Q_i &= -\frac{\Delta Q_j}{1 - f_P}\\
\Delta R_i &= R_i\frac{-\Delta Q_i}{Q_i + \Delta Q_i}\\
\Delta L &= min(-\Delta Q_i f_P, -L)\\
\Delta Q_H &= -\Delta Q_i f_P - \Delta L
\end{align}
$$

## Updating agent state

We have
$$
\Delta r^\alpha_i = -\Delta R_i\\
\Delta r^\alpha_j = -\Delta R_j
$$

## Checking the algebra
### Case 1, $\Delta R_i > 0$ provided

In [1]:
from sympy.solvers import solve
from sympy import Symbol
from sympy import Min

We want
- $Q_i R_i = (Q_i + \Delta Q_i)(R_i + \Delta R_i)$
- $Q_j R_j = (Q_j + \Delta Q_j)(R_j + \Delta R_j)$ when asset fee is zero
- $\Delta Q_i + \Delta Q_j + \Delta Q_H + \Delta L = 0$, i.e. LRNA is entirely accounted for

In [6]:
Qi = Symbol('Qi')
Ri = Symbol('Ri')
Qj = Symbol('Qj')
Rj = Symbol('Rj')
fA = Symbol('fA')
fP = Symbol('fP')
L = Symbol('L')

dRi = Symbol('dRi')
dQi = Qi * (-dRi/(Ri + dRi))
dQj = -dQi * (1 - fP)
dRj = Rj * (-dQj)/(Qj + dQj) * (1 - fA)
dL = Min(-dQi*fP, -L)
dQH = -dQi*fP - dL

print("LRNA accounting")
lhs = dQi + dQj + dQH + dL
rhs = 0
print(lhs.equals(rhs))

print("Swap invariant i")
lhs = Qi * Ri
rhs_fee = (Qi + dQi)*(Ri + dRi)
print(lhs.equals(rhs_fee))

print("Swap invariant j")
lhs = Qj * Rj
rhs_fee = (Qj + dQj)*(Rj + dRj)
rhs_feeless = rhs_fee.subs(fA, 0)
print(lhs.equals(rhs_feeless))

LRNA accounting
True
Swap invariant i
True
Swap invariant j
True


### Case 2, $\Delta R_j < 0$ provided

As before, we want
- $Q_i R_i = (Q_i + \Delta Q_i)(R_i + \Delta R_i)$
- $Q_j R_j = (Q_j + \Delta Q_j)(R_j + \Delta R_j)$ when asset fee is zero
- $\Delta Q_i + \Delta Q_j + \Delta Q_H + \Delta L = 0$, i.e. LRNA is entirely accounted for

We would furthermore like to checkt the fee calculations, namely
- $\Delta Q_j = -\Delta Q_i(1 - f_P)$
- $\Delta R_j = R_j \frac{-\Delta Q_j}{Q_j + \Delta Q_j}(1 - f_A)$

$$
\begin{align}
\Delta Q_j &= Q_j\frac{-\Delta R_j}{R_j(1 - f_A) + \Delta R_j}\\
\Delta Q_i &= -\frac{\Delta Q_j}{1 - f_P}\\
\Delta R_i &= R_i\frac{-\Delta Q_i}{Q_i + \Delta Q_i}\\
\Delta L &= min(-\Delta Q_i f_P, -L)\\
\Delta Q_H &= -\Delta Q_i f_P - \Delta L
\end{align}
$$

In [8]:
Qi = Symbol('Qi')
Ri = Symbol('Ri')
Qj = Symbol('Qj')
Rj = Symbol('Rj')
fA = Symbol('fA')
fP = Symbol('fP')
L = Symbol('L')

dRj = Symbol('dRj')

dQj = Qj * (-dRj)/(Rj * (1-fA) + dRj)
dQi = -dQj/(1 - fP)
dRi = Ri *(-dQi)/(Qi + dQi)
dL = Min(-dQi * fP, -L)
dQH = -dQi * fP - dL

print("LRNA accounting")
lhs = dQi + dQj + dQH + dL
rhs = 0
print(lhs.equals(rhs))

print("Swap invariant i")
lhs = Qi * Ri
rhs = (Qi + dQi)*(Ri + dRi)
print(lhs.equals(rhs))

print("Swap invariant j")
lhs = Qj * Rj
rhs_fee = (Qj + dQj)*(Rj + dRj)
rhs_feeless = rhs_fee.subs(fA, 0)
print(lhs.equals(rhs_feeless))

print("protocol fee")
lhs = dQj
rhs = -dQi * (1 - fP)
print(lhs.equals(rhs))

print("asset fee")
lhs = dRj
rhs = Rj * (-dQj)/(Qj + dQj) * (1 - fA)
print(lhs.equals(rhs))


LRNA accounting
True
Swap invariant i
True
Swap invariant j
True
protocol fee
True
asset fee
True
